In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from joblib import dump, load
import gzip
import numpy as np

In [7]:
df = pd.read_parquet('./data/ETL.parquet')
nombres = pd.read_parquet('./data/NombresJuegos.parquet')

In [3]:
df

,Precio,ItemId,Año_Lanzamiento,Recomendado,Sentimiento,TiempoJugado,Etiquetas_Action,Etiquetas_Adventure,Etiquetas_Casual,Etiquetas_Indie,Etiquetas_Strategy
0,9,282010,1997,1,1,9319,1,0,0,0,0
1,9,282010,1997,1,1,9319,0,0,0,1,0
2,9,70,1998,1,2,2650946,1,0,0,0,0
3,9,70,1998,1,0,2650946,1,0,0,0,0
4,9,70,1998,1,1,2650946,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
14875,9,80,2004,1,0,2469131,0,0,0,0,1
14876,9,80,2004,1,1,2469131,0,1,0,0,0
14877,9,80,2004,0,1,2469131,0,1,0,0,0
14878,9,80,2004,1,2,2469131,0,1,0,0,0


In [4]:
features = df.drop(columns=['ItemId', 'Precio', 'TiempoJugado'])

In [5]:
features

,Año_Lanzamiento,Recomendado,Sentimiento,Etiquetas_Action,Etiquetas_Adventure,Etiquetas_Casual,Etiquetas_Indie,Etiquetas_Strategy
0,1997,1,1,1,0,0,0,0
1,1997,1,1,0,0,0,1,0
2,1998,1,2,1,0,0,0,0
3,1998,1,0,1,0,0,0,0
4,1998,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
14875,2004,1,0,0,0,0,0,1
14876,2004,1,1,0,1,0,0,0
14877,2004,0,1,0,1,0,0,0
14878,2004,1,2,0,1,0,0,0


In [6]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14880 entries, 0 to 14879
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Año_Lanzamiento      14880 non-null  int64
 1   Recomendado          14880 non-null  int64
 2   Sentimiento          14880 non-null  int64
 3   Etiquetas_Action     14880 non-null  int64
 4   Etiquetas_Adventure  14880 non-null  int64
 5   Etiquetas_Casual     14880 non-null  int64
 6   Etiquetas_Indie      14880 non-null  int64
 7   Etiquetas_Strategy   14880 non-null  int64
dtypes: int64(8)
memory usage: 930.1 KB


In [7]:
# Preprocesamiento y normalización de datos
scaler = MinMaxScaler()
df_normalized = scaler.fit_transform(features)

In [8]:

# Cálculo de la similitud de coseno
similarities = cosine_similarity(df_normalized)

In [16]:
similarities

array([[1.        , 0.57111597, 0.96352847, ..., 0.21291461, 0.59711736,
        0.49933291],
       [0.57111597, 1.        , 0.59177339, ..., 0.21291461, 0.59711736,
        0.49933291],
       [0.96352847, 0.59177339, 1.        , ..., 0.307784  , 0.68012131,
        0.44000166],
       ...,
       [0.21291461, 0.21291461, 0.307784  , ..., 1.        , 0.79503711,
        0.68644777],
       [0.59711736, 0.59711736, 0.68012131, ..., 0.79503711, 1.        ,
        0.83412845],
       [0.49933291, 0.49933291, 0.44000166, ..., 0.68644777, 0.83412845,
        1.        ]])

In [10]:
# Calcula la longitud de la primera dimensión de la matriz
longitud = similarities.shape[0]

# Calcula el punto medio para dividir la matriz en dos partes iguales
mitad = longitud // 2

# Divide la matriz en dos partes iguales
parte1 = similarities[:mitad, :]
parte2 = similarities[mitad:, :]

In [11]:
np.save('matriz_similitud_1.npy', parte1)
np.save('matriz_similitud_2.npy', parte2)

In [12]:
with open('matriz_similitud_1.npy', 'rb') as f_in:
    with gzip.open('matriz_similitud_1.npy.gz', 'wb') as f_out:
        f_out.writelines(f_in)

with open('matriz_similitud_2.npy', 'rb') as f_in:
    with gzip.open('matriz_similitud_2.npy.gz', 'wb') as f_out:
        f_out.writelines(f_in)

In [1]:
import gzip
import numpy as np

# Lee la parte 1 desde el archivo comprimido
with gzip.open('matriz_similitud_1.npy.gz', 'rb') as f:
    parte1 = np.load(f)

# Lee la parte 2 desde el archivo comprimido
with gzip.open('matriz_similitud_2.npy.gz', 'rb') as f:
    parte2 = np.load(f)


In [2]:
matriz_reconstruida = np.concatenate((parte1, parte2), axis=0)

In [4]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
# Convertir la matriz NumPy a un DataFrame de pandas
df = pd.DataFrame(matriz_reconstruida)

# Convertir el DataFrame de pandas a un formato compatible con Arrow
table = pa.Table.from_pandas(df)

# Guardar la tabla en un archivo Parquet
pq.write_table(table, 'matriz.parquet')

In [5]:
pq.write_table(table, 'matriz_comprimida.parquet', compression='snappy')

In [3]:
matriz_reconstruida

array([[1.        , 0.57111597, 0.96352847, ..., 0.21291461, 0.59711736,
        0.49933291],
       [0.57111597, 1.        , 0.59177339, ..., 0.21291461, 0.59711736,
        0.49933291],
       [0.96352847, 0.59177339, 1.        , ..., 0.307784  , 0.68012131,
        0.44000166],
       ...,
       [0.21291461, 0.21291461, 0.307784  , ..., 1.        , 0.79503711,
        0.68644777],
       [0.59711736, 0.59711736, 0.68012131, ..., 0.79503711, 1.        ,
        0.83412845],
       [0.49933291, 0.49933291, 0.44000166, ..., 0.68644777, 0.83412845,
        1.        ]])

In [3]:
import gzip
import pyarrow.parquet as pq

# Lee el archivo Parquet
table = pq.read_table('matriz.parquet')

# Divide los datos en dos partes
mitad = len(table) // 2
parte1 = table.slice(0, mitad)
parte2 = table.slice(mitad)

# Guarda cada parte en un archivo Parquet en un buffer de memoria
buffer1 = pa.BufferOutputStream()
buffer2 = pa.BufferOutputStream()
pq.write_table(parte1, buffer1)
pq.write_table(parte2, buffer2)

# Comprime cada buffer con gzip y escribe el resultado en archivos individuales
with gzip.open('parte1.parquet.gz', 'wb') as f:
    f.write(buffer1.getvalue())

with gzip.open('parte2.parquet.gz', 'wb') as f:
    f.write(buffer2.getvalue())


In [1]:
import gzip
import pyarrow.parquet as pq
import pyarrow as pa
import numpy as np

# Leer y descomprimir la parte 1
with gzip.open('parte1.parquet.gz', 'rb') as f:
    buffer1 = f.read()
table1 = pq.read_table(pa.BufferReader(buffer1))

# Leer y descomprimir la parte 2
with gzip.open('parte2.parquet.gz', 'rb') as f:
    buffer2 = f.read()
table2 = pq.read_table(pa.BufferReader(buffer2))

# Concatenar las dos partes en una sola tabla
table_combined = pa.concat_tables([table1, table2])

# Convertir la tabla a un DataFrame de pandas y luego a una matriz de NumPy
df = table_combined.to_pandas()
matriz = df.to_numpy()

# Ahora 'matriz' contendrá los datos de las dos partes del archivo Parquet unidos en una sola matriz de NumPy


In [2]:
matriz

array([[1.        , 0.57111597, 0.96352847, ..., 0.21291461, 0.59711736,
        0.49933291],
       [0.57111597, 1.        , 0.59177339, ..., 0.21291461, 0.59711736,
        0.49933291],
       [0.96352847, 0.59177339, 1.        , ..., 0.307784  , 0.68012131,
        0.44000166],
       ...,
       [0.21291461, 0.21291461, 0.307784  , ..., 1.        , 0.79503711,
        0.68644777],
       [0.59711736, 0.59711736, 0.68012131, ..., 0.79503711, 1.        ,
        0.83412845],
       [0.49933291, 0.49933291, 0.44000166, ..., 0.68644777, 0.83412845,
        1.        ]])

In [8]:
# Función para obtener los ítems más similares dado un ítem de referencia
def obtener_similares(id_item_referencia, n=5):
    index_referencia = df[df['ItemId'] == id_item_referencia].index[0]
    similar_indices = matriz[index_referencia].argsort()[::-1][:n+1]
    similar_items = df.iloc[similar_indices]
    return similar_items

In [9]:
id_item_referencia = 307340  # Id del ítem de referencia
items_similares = obtener_similares(id_item_referencia)
print(items_similares["ItemId"])

14770       220
14704      6980
14762       320
14692      2600
14864     13230
3158     307340
Name: ItemId, dtype: int64


In [10]:
items_similares

,Precio,ItemId,Año_Lanzamiento,Recomendado,Sentimiento,TiempoJugado,Etiquetas_Action,Etiquetas_Adventure,Etiquetas_Casual,Etiquetas_Indie,Etiquetas_Strategy
14770,9,220,2004,1,2,10719468,1,0,0,0,0
14704,8,6980,2004,1,2,136379,1,0,0,0,0
14762,4,320,2004,1,2,2832582,1,0,0,0,0
14692,19,2600,2004,1,2,591896,1,0,0,0,0
14864,14,13230,2004,1,2,345093,1,0,0,0,0
3158,6,307340,2004,1,2,10647,1,0,0,0,0


In [11]:

items_similares['ItemId'] = items_similares['ItemId'].astype(str)
df_resultado = pd.merge(items_similares, nombres, on='ItemId')
df_resultado

/tmp/ipykernel_53564/4229142044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_similares['ItemId'] = items_similares['ItemId'].astype(str)


,Precio,ItemId,Año_Lanzamiento,Recomendado,Sentimiento,TiempoJugado,Etiquetas_Action,Etiquetas_Adventure,Etiquetas_Casual,Etiquetas_Indie,Etiquetas_Strategy,Titulo
0,9,220,2004,1,2,10719468,1,0,0,0,0,Half-Life 2
1,8,6980,2004,1,2,136379,1,0,0,0,0,Thief: Deadly Shadows
2,4,320,2004,1,2,2832582,1,0,0,0,0,Half-Life 2: Deathmatch
3,19,2600,2004,1,2,591896,1,0,0,0,0,Vampire: The Masquerade - Bloodlines
4,14,13230,2004,1,2,345093,1,0,0,0,0,Unreal Tournament 2004: Editor's Choice Edition
5,6,307340,2004,1,2,10647,1,0,0,0,0,Platypus
